## 1.1 Парсинг данных

In [20]:
import pandas as pd
import json
from dateutil import parser
import glob
json_files = [file for file in glob.glob('Desktop/Data/*.json') if '.ipynb_checkpoints' not in file]
data = []

for filename in json_files :
    with open(filename, 'r', encoding='utf-8') as f:
        file_data = json.load(f)
        
        company_info = {
            'name': filename.split('.')[0],  # извлекаем имя компании из имени файла
            'description': file_data.get('info', {}).get('about', '') if file_data.get('info') else None,
            'rating': file_data.get('info', {}).get('rate', 0) if file_data.get('info') else None,
            'industries': ', '.join(file_data.get('info', {}).get('industries', [])) if file_data.get('info') else None,
        }
        
        for ref in file_data.get('refs', []):
            if isinstance(ref, list):
                article_text = ref[0]
                publish_date = parser.parse(' '.join([ref[1]['day'], ref[1]['month'], ref[1]['time']]), fuzzy=True)
                views = ref[2].get('views', 0)
                
                data.append({
                    'company_name': company_info['name'],
                    'description': company_info['description'],
                    'rating': company_info['rating'],
                    'industries': company_info['industries'],
                    'article_text': article_text,
                    'publish_date': publish_date,
                    'views': views
                })

df = pd.DataFrame(data)

загружаем  все данные  с  одной  директории  и  все json  файлы  разбиваем  на  много  колонок  

In [21]:
df # результат  

,company_name,description,rating,industries,article_text,publish_date,views
0,Desktop/Data\Cybersport,None,NaN,None,7 декабря состоялась церемония награждения лау...,2024-05-08 09:30:00,2300
1,Desktop/Data\ESforce Holding,None,NaN,None,7 декабря состоялась церемония награждения лау...,2024-05-08 09:30:00,2300
2,Desktop/Data\ESforce Holding,None,NaN,None,Из совместного исследования издателя компьютер...,2024-05-29 13:50:00,2800
3,Desktop/Data\ESforce Holding,None,NaN,None,По статистике The Esports Observer за I полови...,2024-05-09 09:33:00,6200
4,Desktop/Data\ESforce Holding,None,NaN,None,Компании SuperData Research и PayPal провели с...,2024-05-11 12:43:00,10000
...,...,...,...,...,...,...,...
1107,Desktop/Data\Федерация креативных индустрий,None,NaN,None,"Привет, Хабр! Представляю вашему вниманию пере...",2024-05-30 10:43:00,9300
1108,Desktop/Data\Федерация креативных индустрий,None,NaN,None,Источник\n«Умный человек не делает все ошибки ...,2024-05-24 12:37:00,7700
1109,Desktop/Data\Федерация креативных индустрий,None,NaN,None,"Привет, Хабр! Сегодня мы решили поделиться с с...",2024-05-12 14:09:00,9300
1110,Desktop/Data\Федерация креативных индустрий,None,NaN,None,"Привет! Меня зовут Никита Бокарев, я креативны...",2024-05-04 16:44:00,52000


## 1.2  Формирование структуры набора данных

In [22]:
df.info() # есть нулевые  значения , но в нашем  случае  это не играет роли 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   company_name  1112 non-null   object        
 1   description   293 non-null    object        
 2   rating        293 non-null    float64       
 3   industries    293 non-null    object        
 4   article_text  1112 non-null   object        
 5   publish_date  1112 non-null   datetime64[ns]
 6   views         1112 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 60.9+ KB


In [6]:
df.fillna('нет  информации' , inplace = True) # заполним  пропущеные  значения 

C:\Users\123\AppData\Local\Temp\ipykernel_12996\3573407944.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'нет  информации' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('нет  информации' , inplace = True)


оставим все   признаки для дальнейшей кластеризации так как у нас и так мало признаков  

## 1.3 Предварительная обработка текстовых данных

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\123\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\123\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\123\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\123\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\123\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\123\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

нужно избавиться от  всех  спец символов  привести слова к  одной  форме

будем  использовать  для  этого   билиотеку  ntkl  так как  у неё  есть хорошая  докуменатции и она оптимизирована 

In [18]:
def preprocess_text(text):
    # Токенизация
    tokens = word_tokenize(text.lower())

    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Выделение значимых частей речи

    # Удаление стоп-слов
    stop_words = set(stopwords.words('russian'))
    filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words]

    # Удаление пунктуации и спецсимволов
    filtered_tokens = [token for token in filtered_tokens if token.isalpha()]

    # Удаление редких слов
    word_counts = Counter(filtered_tokens)
    rare_words = set(word for word, count in word_counts.items() if count < 5)
    preprocessed_tokens = [token for token in filtered_tokens if token not in rare_words]

    return ' '.join(preprocessed_tokens)

df['preprocessed_text'] = df['article_text'].apply(preprocess_text)

df['preprocessed_text'].head()

0    рунета номинации номинации рунета номинации но...
1    рунета номинации номинации рунета номинации но...
2    world of of of world of world of world of worl...
3    млн dota ruhub dota млн млн млн млн млн млн do...
4    superdata research киберспорта киберспорта sup...
Name: preprocessed_text, dtype: object

In [19]:
df['preprocessed_text'][0]

'рунета номинации номинации рунета номинации номинация москвы проект москвы номинация номинация москвы проект проект проект номинация номинация проект номинация москвы номинация проект номинация проект москвы проект номинация номинация проект проект номинация номинация проект проект москвы москвы проект номинация рунета номинации номинации номинация рунета рунета номинации'

In [11]:
df

,company_name,description,rating,industries,article_text,publish_date,views,preprocessed_text
0,Desktop/Data\Cybersport,нет информации,нет информации,нет информации,7 декабря состоялась церемония награждения лау...,2024-05-08 09:30:00,2300,рунета номинации номинации рунета номинации но...
1,Desktop/Data\ESforce Holding,нет информации,нет информации,нет информации,7 декабря состоялась церемония награждения лау...,2024-05-08 09:30:00,2300,рунета номинации номинации рунета номинации но...
2,Desktop/Data\ESforce Holding,нет информации,нет информации,нет информации,Из совместного исследования издателя компьютер...,2024-05-29 13:50:00,2800,world of of of world of world of world of worl...
3,Desktop/Data\ESforce Holding,нет информации,нет информации,нет информации,По статистике The Esports Observer за I полови...,2024-05-09 09:33:00,6200,млн dota ruhub dota млн млн млн млн млн млн do...
4,Desktop/Data\ESforce Holding,нет информации,нет информации,нет информации,Компании SuperData Research и PayPal провели с...,2024-05-11 12:43:00,10000,superdata research киберспорта киберспорта sup...
...,...,...,...,...,...,...,...,...
1107,Desktop/Data\Федерация креативных индустрий,нет информации,нет информации,нет информации,"Привет, Хабр! Представляю вашему вниманию пере...",2024-05-30 10:43:00,9300,technolog cooper tech technolog cooper многих ...
1108,Desktop/Data\Федерация креативных индустрий,нет информации,нет информации,нет информации,Источник\n«Умный человек не делает все ошибки ...,2024-05-24 12:37:00,7700,источник цифровое предприятие предприятий цифр...
1109,Desktop/Data\Федерация креативных индустрий,нет информации,нет информации,нет информации,"Привет, Хабр! Сегодня мы решили поделиться с с...",2024-05-12 14:09:00,9300,работе дизайн государственных систем алина гов...
1110,Desktop/Data\Федерация креативных индустрий,нет информации,нет информации,нет информации,"Привет! Меня зовут Никита Бокарев, я креативны...",2024-05-04 16:44:00,52000,киберспорт киберспорта киберспорта мира киберс...


In [12]:
df.to_csv('articles1.csv')

мы  провели   предварительную обработку текста с помощью методов NLP: токенизацию, 
лемматизацию,, а также удаление стоп-слов, пунктуации, 
спецсимволовх.

просто разобраться  

In [23]:
df

,company_name,description,rating,industries,article_text,publish_date,views
0,Desktop/Data\Cybersport,None,NaN,None,7 декабря состоялась церемония награждения лау...,2024-05-08 09:30:00,2300
1,Desktop/Data\ESforce Holding,None,NaN,None,7 декабря состоялась церемония награждения лау...,2024-05-08 09:30:00,2300
2,Desktop/Data\ESforce Holding,None,NaN,None,Из совместного исследования издателя компьютер...,2024-05-29 13:50:00,2800
3,Desktop/Data\ESforce Holding,None,NaN,None,По статистике The Esports Observer за I полови...,2024-05-09 09:33:00,6200
4,Desktop/Data\ESforce Holding,None,NaN,None,Компании SuperData Research и PayPal провели с...,2024-05-11 12:43:00,10000
...,...,...,...,...,...,...,...
1107,Desktop/Data\Федерация креативных индустрий,None,NaN,None,"Привет, Хабр! Представляю вашему вниманию пере...",2024-05-30 10:43:00,9300
1108,Desktop/Data\Федерация креативных индустрий,None,NaN,None,Источник\n«Умный человек не делает все ошибки ...,2024-05-24 12:37:00,7700
1109,Desktop/Data\Федерация креативных индустрий,None,NaN,None,"Привет, Хабр! Сегодня мы решили поделиться с с...",2024-05-12 14:09:00,9300
1110,Desktop/Data\Федерация креативных индустрий,None,NaN,None,"Привет! Меня зовут Никита Бокарев, я креативны...",2024-05-04 16:44:00,52000


In [35]:
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint
import json
from tqdm import tqdm
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models  
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import rcParams
%matplotlib inline
import nltk
import pymorphy2
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

In [28]:
STOPWORDS_DIR = "stop"

In [29]:
def get_stop_words():
    stopwordsrus = list(stopwords.words('russian'))
    with open(os.path.join(STOPWORDS_DIR, "russian.txt")) as f:
        stop_words_1 = f.read().split("\n")
    with open(os.path.join(STOPWORDS_DIR, "stopwords-ru.txt")) as f:
        stop_words_2 = f.read().split("\n")
    return list(set(stopwordsrus+stop_words_1+stop_words_2))

In [30]:
stopwordsru = get_stop_words()

In [31]:
def process(text):
    return list(t.lower() for t in word_tokenize(text) if t.isalpha() and t.lower() not in stopwordsru)

In [33]:
data = [process(t) for t in df['article_text']]

In [36]:
morph = pymorphy2.MorphAnalyzer()
def lemmatizer(texts):
    return [[morph.parse(word)[0] for word in text] for text in texts]

In [37]:
morph_data = lemmatizer(data)

In [38]:
def extract_lemma(texts):
    norm = []
    for t in texts:
        res = []
        for word in t:
            n = word.normal_form
            res.append(n)
        norm.append(res)
    return norm

In [39]:
data_norm = extract_lemma(morph_data)

In [46]:
df['tokenized_text'] = data_norm

In [48]:
df.to_csv('articles_full.csv')